## What is Flexible Querying?

Flexible querying provides the ability to execute a performant query that spans multiple indexes in your data store. This means you can write ad-hoc, dynamically generated queries, where you don't need to know the query, fields or ordering of fields in advance.

## Your application is in a constant state of evolution

Let’s say we have a hypothetical movie application with documents like:

In [ ]:
import pymongo
# from config import mongo_uri
from pymongoexplain import ExplainableCollection
import pprint

pp = pprint.PrettyPrinter()

doc = {
  "title": "Fight Club",
  "year": 1999,
  "imdb": {
    "rating": 8.9,
    "votes": 1191784,
    "id": 137523
  },
  "cast":[
    "Edward Norton",
    "Brad Pitt"
  ]
}

# connect
conn = pymongo.MongoClient("mongo_uri")
collection = conn['sample_mflix']['movies']

# insert
collection.insert_one(doc)


## Initial Product Requirements

Now for the version 1.0 requirements, you need to query on title & year, so you first create a compound index:

In [ ]:
collection.create_index([
    ('title', 1),
    ('year', 1)
], name='title_year', default_language='english')

In [ ]:
pipeline = [
    {"$match":{"title":"Fight Club", "year":1999} }
]

explain_plan = ExplainableCollection(
    collection, 
    verbosity="executionStats"
).aggregate(pipeline)

pp.pprint(explain_plan['executionStats']['executionStages'])

## Our query evolves

Now our application requirements have evolved and you need to query on cast and imdb. First you create the index, then issue the query:

In [ ]:
collection.create_index([
    ('cast', 1),
    ('imdb.rating', 1)
], name='cast_rating', default_language='english')

In [ ]:
pipeline = [
    {"$match":{"cast":"Edward Norton", "imdb.rating":{ "$gte":9 } } }
]

explain_plan = ExplainableCollection(
    collection, 
    verbosity="executionStats"
).aggregate(pipeline)

pp.pprint(explain_plan['executionStats']['executionStages'])

## And our query retracts

Now, our application requires you issue a new query, a subset of the original:

In [ ]:
pipeline = [
    {"$match":{"imdb.rating" : { "$gte":9 } } }
]

explain_plan = ExplainableCollection(
    collection, 
    verbosity="executionStats"
).aggregate(pipeline)

pp.pprint(explain_plan['executionStats']['executionStages'])

The query above results in the dreaded collection scan despite the previous compound index (cast_imdb.rating) comprising the above query’s key. 

__Note: Collection scans should be avoided because not only do they instruct the cursor to look at every document in the collection which is slow, but it also forces documents out of memory resulting in increased I/O pressure.__

Now you certainly could create a new index composed of just imdb.rating, which would return an index scan for the above query,  but that’s three different indexes that the query planner would have to navigate in order to select the most performant response.

## Alternatively: Atlas Search

Because Lucene uses a different index data structure (inverted indexes vs B-tree indexes) it’s purpose-built to handle flexible querying well.

If you create a single index that maps all of our 4 fields above (title, year, cast, imdb):


In [ ]:
{
  "mappings": {
    "dynamic": False,
    "fields": {
      "title": {
        "type": "string",
        "dynamic": False
      },
      "year": {
        "type": "number",
        "dynamic": False
      },
      "cast": {
        "type": "string",
        "dynamic": False
      },
      "imdb.rating": {
        "type": "number",
        "dynamic": False
      }                  
    }
  }
}

Then you issue a query that first spans title & year via a must (AND) clause, which is the equivalent of  `db.collection.find({"title":"Fight Club", "year":1999})`:

In [ ]:
pipeline = [{
  "$search": {
    "compound": {
      "must": [{
          "text": {
            "query": "Fight Club",
            "path": "title"
          }
        },
        {
          "range": {
            "path": "year",
            "gte": 1999,
            "lte": 1999
          }
        }
      ]
    }
  }
}]

explain_plan = ExplainableCollection(
    collection, 
    verbosity="allPlansExecution"
).aggregate(pipeline)

# pp.pprint(explain_plan)

Then when you add `imdb` and `cast` to the query, you can still get performant results:

In [ ]:
pipeline = [{
    "$search": {
      "compound": {
        "must": [
          {
            "text": {
              "query": "Fight",
              "path": "title"
            }
          },
          {
            "range": {
              "path": "year",
              "gte": 1999,
              "lte": 1999
            }
          },
            {
              "text": {
                "query": "Edward Norton",
                "path": "cast"
              }
            },
            {
              "range": {
                "gte": 9,
                "path": "imdb.rating"
              }
            }
          ]
        }
      }
    }
  ]
            
        
explain_plan = ExplainableCollection(
    collection, 
    verbosity="allPlansExecution"
).aggregate(pipeline)

# pp.pprint(explain_plan)

[Link to blog post](https://www.mongodb.com/developer/products/atlas/atlas-search-for-index-intersection/)